In [2]:
# import dependencies
import pandas as pd
import numpy as np
import re

In [3]:
# load in csv files
world_cups_csv = "Data/WorldCups.csv"
wc_matches_csv = "Data/WorldCupMatches.csv"
wc_players_csv = "Data/WorldCupPlayers.csv"

In [4]:
# read the csv files to data frames
world_cups_df = pd.read_csv(world_cups_csv)
wc_matches_df = pd.read_csv(wc_matches_csv)
wc_players_df = pd.read_csv(wc_players_csv)

### World Cups Data

In [5]:
world_cups_df['Attendance'] = world_cups_df['Attendance'].str.replace('.','').astype(int)

In [6]:
world_cups_df

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1045246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768607
5,1958,Sweden,Brazil,Sweden,France,Germany FR,126,16,35,819810
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,893172
7,1966,England,England,Germany FR,Portugal,Soviet Union,89,16,32,1563135
8,1970,Mexico,Brazil,Italy,Germany FR,Uruguay,95,16,32,1603975
9,1974,Germany,Germany FR,Netherlands,Poland,Brazil,97,16,38,1865753


### World Cup Matches

In [7]:
wc_matches_df = wc_matches_df.dropna()

In [8]:
wc_matches_df = wc_matches_df.apply(pd.to_numeric, errors='ignore', downcast='integer')

In [9]:
wc_matches_df

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,4444,3,0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201,1096,FRA,MEX
1,1930,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3,0,Belgium,,18346,2,0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201,1090,USA,BEL
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,24059,2,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201,1093,YUG,BRA
3,1930,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3,1,Peru,,2549,1,0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201,1098,ROU,PER
4,1930,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1,0,France,,23409,0,0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201,1085,ARG,FRA
5,1930,16 Jul 1930 - 14:45,Group 1,Parque Central,Montevideo,Chile,3,0,Mexico,,9249,1,0,CRISTOPHE Henry (BEL),APHESTEGUY Martin (URU),LANGENUS Jean (BEL),201,1095,CHI,MEX
6,1930,17 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,4,0,Bolivia,,18306,0,0,MATEUCCI Francisco (URU),LOMBARDI Domingo (URU),WARNKEN Alberto (CHI),201,1092,YUG,BOL
7,1930,17 Jul 1930 - 14:45,Group 4,Parque Central,Montevideo,USA,3,0,Paraguay,,18306,2,0,MACIAS Jose (ARG),APHESTEGUY Martin (URU),TEJADA Anibal (URU),201,1097,USA,PAR
8,1930,18 Jul 1930 - 14:30,Group 3,Estadio Centenario,Montevideo,Uruguay,1,0,Peru,,57735,0,0,LANGENUS Jean (BEL),BALWAY Thomas (FRA),CRISTOPHE Henry (BEL),201,1099,URU,PER
9,1930,19 Jul 1930 - 12:50,Group 1,Estadio Centenario,Montevideo,Chile,1,0,France,,2000,0,0,TEJADA Anibal (URU),LOMBARDI Domingo (URU),REGO Gilberto (BRA),201,1094,CHI,FRA


### World Cup Players

In [10]:
del wc_players_df['Coach Name']
del wc_players_df['Line-up']
del wc_players_df['Shirt Number']
del wc_players_df['Position']

In [11]:
wc_players_df['Event'] = wc_players_df['Event'].fillna('')

In [15]:
wc_players_df

,RoundID,MatchID,Team Initials,Player Name,Event
0,201,1096,FRA,Alex THEPOT,
1,201,1096,MEX,Oscar BONFIGLIO,
2,201,1096,FRA,Marcel LANGILLER,G40'
3,201,1096,MEX,Juan CARRENO,G70'
4,201,1096,FRA,Ernest LIBERATI,
5,201,1096,MEX,Rafael GARZA,
6,201,1096,FRA,Andre MASCHINOT,G43' G87'
7,201,1096,MEX,Hilario LOPEZ,
8,201,1096,FRA,Etienne MATTLER,
9,201,1096,MEX,Dionisio MEJIA,


In [13]:
unique_players_df = wc_players_df.drop_duplicates(['Player Name']).reset_index(drop=True)

In [14]:
unique_players_df

,RoundID,MatchID,Team Initials,Player Name,Event
0,201,1096,FRA,Alex THEPOT,
1,201,1096,MEX,Oscar BONFIGLIO,
2,201,1096,FRA,Marcel LANGILLER,G40'
3,201,1096,MEX,Juan CARRENO,G70'
4,201,1096,FRA,Ernest LIBERATI,
5,201,1096,MEX,Rafael GARZA,
6,201,1096,FRA,Andre MASCHINOT,G43' G87'
7,201,1096,MEX,Hilario LOPEZ,
8,201,1096,FRA,Etienne MATTLER,
9,201,1096,MEX,Dionisio MEJIA,
